# Notebook 4: Advanced Retrieval

**Difficulty:** Intermediate-Advanced | **Estimated Time:** 120-150 minutes

## Learning Objectives

By the end of this notebook, you will be able to:

1. ✅ Implement RecursiveRetriever for hierarchical document retrieval
2. ✅ Use QueryFusionRetriever to combine multiple retrieval strategies
3. ✅ Build custom retrievers with specialized ranking logic
4. ✅ Apply query transformation techniques (HyDE, rewriting, multi-query)
5. ✅ Implement complex metadata filtering (AND/OR/NOT logic)
6. ✅ Evaluate retrieval quality with metrics

## Prerequisites

- Completed Notebooks 1, 2, and 3
- Understanding of embeddings and vector similarity
- Familiarity with basic retrievers

## Curriculum Coverage

- **Section 3.5.3-3.5.5:** RecursiveRetriever, QueryFusionRetriever, Custom Retrievers
- **Section 3.6:** Query Optimization
- **Section 4.3:** Query Engines with Filters
- **Section 4.4:** Advanced Retrieval Strategies

---

## 1. Setup & Imports

In [1]:
# Core LlamaIndex
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    Document,
    SummaryIndex,
)
from llama_index.core.schema import TextNode, IndexNode, NodeWithScore
from llama_index.core.retrievers import (
    VectorIndexRetriever,
    RecursiveRetriever,
    QueryFusionRetriever,
    BaseRetriever,
)
from llama_index.core.query_engine import RetrieverQueryEngine

# Query transformations
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

# Metadata filtering
from llama_index.core.vector_stores import (
    MetadataFilters,
    MetadataFilter,
    FilterOperator,
    FilterCondition,
)

# LLM and Embeddings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Utilities
from dotenv import load_dotenv
import os
from typing import List, Optional
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports successful!")

✅ Imports successful!


In [2]:
# Configure Settings
load_dotenv()

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    dimensions=1536
)

print("✅ Settings configured")

✅ Settings configured


---

## 2. Prepare Hierarchical Research Paper Data

In [3]:
# Create research papers with hierarchical structure
papers_data = [
    {
        "title": "Attention Is All You Need",
        "authors": "Vaswani et al.",
        "year": 2017,
        "citations": 85000,
        "category": "transformers",
        "abstract": """We propose a new simple network architecture, the Transformer, based solely 
        on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments show 
        that the Transformer achieves state-of-the-art results in machine translation while being more 
        parallelizable and requiring significantly less time to train.""",
        "sections": [
            {
                "name": "Introduction",
                "content": """Recurrent neural networks have been the dominant approach for sequence modeling. 
                However, they are inherently sequential and cannot be parallelized. The Transformer is the 
                first transduction model relying entirely on self-attention to compute representations."""
            },
            {
                "name": "Model Architecture",
                "content": """The Transformer follows an encoder-decoder structure. The encoder maps an input 
                sequence to a sequence of continuous representations. The decoder generates an output sequence 
                one element at a time. Both use stacked self-attention and point-wise fully connected layers."""
            },
            {
                "name": "Self-Attention",
                "content": """Self-attention allows the model to attend to different positions of the input 
                sequence when encoding each position. We use scaled dot-product attention and multi-head 
                attention mechanisms. This enables modeling of dependencies regardless of distance."""
            },
        ]
    },
    {
        "title": "BERT: Pre-training of Deep Bidirectional Transformers",
        "authors": "Devlin et al.",
        "year": 2019,
        "citations": 65000,
        "category": "language_models",
        "abstract": """BERT is designed to pre-train deep bidirectional representations from unlabeled text 
        by jointly conditioning on both left and right context. Pre-trained BERT can be fine-tuned with 
        just one additional output layer for a wide range of tasks.""",
        "sections": [
            {
                "name": "Introduction",
                "content": """Language model pre-training has been shown to be effective for many NLP tasks. 
                BERT alleviates the unidirectionality constraint by using a masked language model (MLM) 
                pre-training objective, which randomly masks tokens and predicts them."""
            },
            {
                "name": "Pre-training Tasks",
                "content": """BERT uses two pre-training tasks: Masked Language Model (MLM) and Next Sentence 
                Prediction (NSP). MLM masks 15% of tokens randomly and predicts them. NSP predicts whether 
                two sentences follow each other in the original text."""
            },
            {
                "name": "Fine-tuning",
                "content": """For sequence-level tasks, we take the final hidden state of the [CLS] token. 
                For token-level tasks like NER, we take the final hidden state of each token. Fine-tuning 
                is straightforward and requires minimal task-specific modifications."""
            },
        ]
    },
    {
        "title": "Retrieval-Augmented Generation (RAG)",
        "authors": "Lewis et al.",
        "year": 2020,
        "citations": 3500,
        "category": "rag",
        "abstract": """We explore RAG models which combine parametric and non-parametric memory for 
        language generation. RAG models use a dense vector index of Wikipedia as non-parametric memory 
        and a pre-trained seq2seq transformer as parametric memory.""",
        "sections": [
            {
                "name": "Introduction",
                "content": """Pre-trained language models store knowledge in parameters but struggle with 
                knowledge-intensive tasks. RAG provides access to external knowledge through retrieval, 
                allowing more interpretable and modular systems."""
            },
            {
                "name": "RAG Architecture",
                "content": """RAG has two components: a retriever based on DPR (Dense Passage Retrieval) and 
                a generator based on BART. The retriever finds relevant documents, which are concatenated 
                with the input and fed to the generator."""
            },
            {
                "name": "Training",
                "content": """We train RAG end-to-end by marginalizing over retrieved documents. The retriever 
                and generator are jointly optimized. This allows the model to learn which documents are most 
                useful for generation."""
            },
        ]
    },
]

print(f"✅ Created {len(papers_data)} research papers with hierarchical structure")
for paper in papers_data:
    print(f"  - {paper['title']} ({paper['year']}) - {len(paper['sections'])} sections")

✅ Created 3 research papers with hierarchical structure
  - Attention Is All You Need (2017) - 3 sections
  - BERT: Pre-training of Deep Bidirectional Transformers (2019) - 3 sections
  - Retrieval-Augmented Generation (RAG) (2020) - 3 sections


---

## 3. RecursiveRetriever: Hierarchical Document Structure

### Why RecursiveRetriever?

For documents with **hierarchical structure** (papers with sections, books with chapters):
- **Top-level nodes**: Summaries or abstracts
- **Detail nodes**: Full sections or chapters
- **RecursiveRetriever**: Retrieves top-level, then drills down to details

### Building Hierarchical Indexes

In [4]:
# Build hierarchical structure
all_nodes = []
index_nodes = []  # Top-level summary nodes
retriever_dict = {}

for paper in papers_data:
    # Create nodes for each section
    section_nodes = []
    for section in paper["sections"]:
        node = TextNode(
            text=section["content"],
            metadata={
                "title": paper["title"],
                "section": section["name"],
                "year": paper["year"],
                "category": paper["category"],
                "citations": paper["citations"],
            }
        )
        section_nodes.append(node)
        all_nodes.append(node)
    
    # Create vector index for this paper's sections
    paper_index = VectorStoreIndex(section_nodes)

    # Store retriever in dictionary (NOT in index_node.obj)
    retriever_dict[paper["title"]] = paper_index.as_retriever(similarity_top_k=2)
    
    # Create summary node (IndexNode) pointing to the paper's index
    index_node = IndexNode(
        text=paper["abstract"],
        index_id=paper["title"],
        metadata={
            "title": paper["title"],
            "authors": paper["authors"],
            "year": paper["year"],
            "category": paper["category"],
            "citations": paper["citations"],
            "type": "summary",
        },
    )
    #index_node.obj = paper_index.as_retriever(similarity_top_k=2)
    index_nodes.append(index_node)

print(f"✅ Created hierarchical structure:")
print(f"   Top-level summary nodes: {len(index_nodes)}")
print(f"   Detail section nodes: {len(all_nodes)}")

✅ Created hierarchical structure:
   Top-level summary nodes: 3
   Detail section nodes: 9


In [5]:
print(retriever_dict.keys())

dict_keys(['Attention Is All You Need', 'BERT: Pre-training of Deep Bidirectional Transformers', 'Retrieval-Augmented Generation (RAG)'])


In [6]:
# Create top-level index of summaries
top_level_index = VectorStoreIndex(index_nodes)

for index_node in index_nodes:
    paper_title = index_node.index_id
    index_node.obj = retriever_dict[paper_title]


# Add the top-level retriever to the dictionary
retriever_dict["vector"] = top_level_index.as_retriever(similarity_top_k=2)

# Create recursive retriever
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict=retriever_dict,
    verbose=True,
)

print("✅ RecursiveRetriever created")

✅ RecursiveRetriever created


In [7]:
# Query with recursive retriever
query = "How does the Transformer architecture handle sequential data?"

print(f"Query: {query}\n")
print("RecursiveRetriever will:")
print("  1. Search top-level summaries (abstracts)")
print("  2. Retrieve detailed sections from matching papers\n")

nodes = recursive_retriever.retrieve(query)

print(f"\nRetrieved {len(nodes)} nodes:\n")
for i, node in enumerate(nodes, 1):
    print(f"Node {i}:")
    print(f"  Title: {node.metadata.get('title')}")
    print(f"  Section: {node.metadata.get('section', 'N/A')}")
    print(f"  Score: {node.score:.4f}")
    print(f"  Text (first 150 chars): {node.text[:150]}...")
    print()

Query: How does the Transformer architecture handle sequential data?

RecursiveRetriever will:
  1. Search top-level summaries (abstracts)
  2. Retrieve detailed sections from matching papers

Retrieving with query id None: How does the Transformer architecture handle sequential data?
Retrieved node with id, entering: Attention Is All You Need
Retrieving with query id Attention Is All You Need: How does the Transformer architecture handle sequential data?
Retrieving text node: The Transformer follows an encoder-decoder structure. The encoder maps an input 
                sequence to a sequence of continuous representations. The decoder generates an output sequence 
                one element at a time. Both use stacked self-attention and point-wise fully connected layers.
Retrieving text node: Recurrent neural networks have been the dominant approach for sequence modeling. 
                However, they are inherently sequential and cannot be parallelized. The Transformer is the 
   

### RecursiveRetriever Benefits

**Advantages:**
- ✅ **Efficient for large documents**: First filter by summary, then get details
- ✅ **Better precision**: Two-stage retrieval reduces noise
- ✅ **Flexible structure**: Supports arbitrary depth hierarchies

**Use cases:**
- Research papers (abstract → sections)
- Books (chapters → subsections)
- Documentation (overview → detailed guides)
- Legal documents (summary → clauses)

---

## 4. QueryFusionRetriever: Combining Multiple Strategies

### Why Query Fusion?

Different retrievers excel at different tasks:
- **Vector retriever**: Semantic similarity
- **Keyword retriever**: Exact term matching
- **Metadata retriever**: Structured filtering

**QueryFusionRetriever** combines multiple retrievers using **Reciprocal Rank Fusion (RRF)**

In [8]:
# Create a flat index for fusion experiments
flat_documents = []
for paper in papers_data:
    # Add abstract
    flat_documents.append(Document(
        text=f"{paper['title']}\n\nAbstract: {paper['abstract']}",
        metadata={
            "title": paper["title"],
            "year": paper["year"],
            "category": paper["category"],
            "citations": paper["citations"],
            "type": "abstract"
        }
    ))
    # Add sections
    for section in paper["sections"]:
        flat_documents.append(Document(
            text=f"{paper['title']} - {section['name']}\n\n{section['content']}",
            metadata={
                "title": paper["title"],
                "section": section["name"],
                "year": paper["year"],
                "category": paper["category"],
                "type": "section"
            }
        ))

fusion_index = VectorStoreIndex.from_documents(flat_documents)
print(f"✅ Created flat index with {len(flat_documents)} documents")

✅ Created flat index with 12 documents


In [9]:
# Create multiple retrievers with different strategies

# Retriever 1: High similarity threshold (precision-focused)
precision_retriever = VectorIndexRetriever(
    index=fusion_index,
    similarity_top_k=5,
)

# Retriever 2: Lower threshold, more results (recall-focused)
recall_retriever = VectorIndexRetriever(
    index=fusion_index,
    similarity_top_k=8,
)

# Create QueryFusionRetriever
fusion_retriever = QueryFusionRetriever(
    retrievers=[precision_retriever, recall_retriever],
    similarity_top_k=5,  # Final number of results
    num_queries=1,  # Generate 1 query variant
    mode="reciprocal_rerank",  # Use Reciprocal Rank Fusion
    use_async=False,
)

print("✅ QueryFusionRetriever created with 2 retrievers")

✅ QueryFusionRetriever created with 2 retrievers


In [10]:
# Query with fusion retriever
fusion_query = "What are the key innovations in BERT?"

print(f"Query: {fusion_query}\n")
fusion_nodes = fusion_retriever.retrieve(fusion_query)

print(f"Retrieved {len(fusion_nodes)} fused results:\n")
for i, node in enumerate(fusion_nodes, 1):
    print(f"Node {i}:")
    print(f"  Title: {node.metadata.get('title')}")
    print(f"  Type: {node.metadata.get('type')}")
    print(f"  Section: {node.metadata.get('section', 'N/A')}")
    print(f"  Score: {node.score:.4f}")
    print()

Query: What are the key innovations in BERT?

Retrieved 5 fused results:

Node 1:
  Title: BERT: Pre-training of Deep Bidirectional Transformers
  Type: abstract
  Section: N/A
  Score: 0.0333

Node 2:
  Title: BERT: Pre-training of Deep Bidirectional Transformers
  Type: section
  Section: Introduction
  Score: 0.0328

Node 3:
  Title: BERT: Pre-training of Deep Bidirectional Transformers
  Type: section
  Section: Fine-tuning
  Score: 0.0323

Node 4:
  Title: BERT: Pre-training of Deep Bidirectional Transformers
  Type: section
  Section: Pre-training Tasks
  Score: 0.0317

Node 5:
  Title: Attention Is All You Need
  Type: abstract
  Section: N/A
  Score: 0.0312



### 🎯 ML Engineering Note: Reciprocal Rank Fusion (RRF)

**How RRF Works:**

For each document, RRF score = Σ(1 / (k + rank_i))
- `k`: Constant (typically 60)
- `rank_i`: Document's rank in retriever i

**Example:**
- Doc A: Rank 1 in Retriever 1, Rank 3 in Retriever 2
- RRF score = 1/(60+1) + 1/(60+3) = 0.0164 + 0.0159 = 0.0323

**Benefits:**
- ✅ No normalization needed (rank-based, not score-based)
- ✅ Robust to different scoring scales
- ✅ Favors documents that rank well in multiple retrievers
- ✅ Simple and effective

---

## 5. Custom Retriever Implementation

### Building a Citation-Aware Retriever

In [11]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import QueryBundle

class CitationBoostRetriever(BaseRetriever):
    """Custom retriever that boosts highly-cited papers."""
    
    def __init__(
        self,
        base_retriever: BaseRetriever,
        citation_weight: float = 0.3,
    ):
        """Initialize with base retriever and citation weight.
        
        Args:
            base_retriever: Underlying vector retriever
            citation_weight: How much to boost based on citations (0-1)
        """
        self._base_retriever = base_retriever
        self._citation_weight = citation_weight
        super().__init__()
    
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes and re-rank by citations."""
        # Get base results
        nodes = self._base_retriever.retrieve(query_bundle)
        
        # Re-rank with citation boost
        max_citations = max(
            node.metadata.get("citations", 0) for node in nodes
        ) or 1
        
        for node in nodes:
            citations = node.metadata.get("citations", 0)
            citation_score = citations / max_citations
            
            # Combine similarity score with citation boost
            original_score = node.score or 0
            node.score = (
                (1 - self._citation_weight) * original_score +
                self._citation_weight * citation_score
            )
        
        # Re-sort by new scores
        nodes.sort(key=lambda x: x.score, reverse=True)
        
        return nodes

print("✅ CitationBoostRetriever class defined")

✅ CitationBoostRetriever class defined


In [12]:
# Create base retriever
base_retriever = fusion_index.as_retriever(similarity_top_k=5)

# Create citation-aware retriever
citation_retriever = CitationBoostRetriever(
    base_retriever=base_retriever,
    citation_weight=0.3,  # 30% weight to citations
)

# Compare results
test_query = "machine learning models"

print(f"Query: {test_query}\n")
print("=" * 80)

# Base retriever results
base_nodes = base_retriever.retrieve(test_query)
print("\nBase Retriever (similarity only):")
for i, node in enumerate(base_nodes[:3], 1):
    print(f"  {i}. {node.metadata.get('title')} - Score: {node.score:.4f}, Citations: {node.metadata.get('citations', 0)}")

# Citation-boosted results
citation_nodes = citation_retriever.retrieve(test_query)
print("\nCitation-Boosted Retriever:")
for i, node in enumerate(citation_nodes[:3], 1):
    print(f"  {i}. {node.metadata.get('title')} - Score: {node.score:.4f}, Citations: {node.metadata.get('citations', 0)}")

Query: machine learning models


Base Retriever (similarity only):
  1. Retrieval-Augmented Generation (RAG) - Score: 0.3737, Citations: 0
  2. BERT: Pre-training of Deep Bidirectional Transformers - Score: 0.3628, Citations: 0
  3. Attention Is All You Need - Score: 0.3554, Citations: 0

Citation-Boosted Retriever:
  1. Retrieval-Augmented Generation (RAG) - Score: 0.2616, Citations: 0
  2. BERT: Pre-training of Deep Bidirectional Transformers - Score: 0.2540, Citations: 0
  3. Attention Is All You Need - Score: 0.2488, Citations: 0


### Custom Retriever Use Cases

**When to build custom retrievers:**

1. **Domain-specific ranking**: Boost based on recency, authority, citations
2. **Business logic**: Apply pricing tiers, user permissions, content policies
3. **Hybrid scoring**: Combine multiple signals (semantic + BM25 + custom)
4. **A/B testing**: Experiment with ranking algorithms
5. **Diversity**: Ensure results cover different aspects/sources

---

## 6. Query Transformation: HyDE

### What is HyDE?

**HyDE (Hypothetical Document Embeddings)**:
1. LLM generates a hypothetical answer to the query
2. Embed the hypothetical answer (not the query)
3. Retrieve documents similar to the hypothetical answer

**Why?** Hypothetical answers are often more similar to real documents than queries.

In [13]:
# Create HyDE query transform
hyde = HyDEQueryTransform(include_original=True)

# Create base query engine
base_query_engine = fusion_index.as_query_engine(similarity_top_k=3)

# Wrap with HyDE transform
hyde_query_engine = TransformQueryEngine(
    base_query_engine,
    query_transform=hyde,
)

print("✅ HyDE query engine created")

✅ HyDE query engine created


In [14]:
# Test HyDE vs standard retrieval
hyde_query = "What are the limitations of recurrent neural networks?"

print(f"Query: {hyde_query}\n")
print("=" * 80)

# Standard query
print("\nStandard Query Engine:")
standard_response = base_query_engine.query(hyde_query)
print(f"Response: {standard_response}")

print("\n" + "=" * 80)

# HyDE query
print("\nHyDE Query Engine:")
print("(First generates hypothetical answer, then retrieves)")
hyde_response = hyde_query_engine.query(hyde_query)
print(f"Response: {hyde_response}")

Query: What are the limitations of recurrent neural networks?


Standard Query Engine:
Response: Recurrent neural networks are inherently sequential, which means they cannot be parallelized. This limitation affects their efficiency and speed in processing sequences.


HyDE Query Engine:
(First generates hypothetical answer, then retrieves)
Response: Recurrent neural networks are inherently sequential, which means they cannot be parallelized. This limitation affects their efficiency and scalability in sequence modeling tasks.


### 🎯 ML Engineering Note: HyDE Trade-offs

**Pros:**
- ✅ Better retrieval for complex queries
- ✅ Handles vocabulary mismatch (query terms ≠ document terms)
- ✅ Effective for knowledge-intensive questions

**Cons:**
- ❌ Extra LLM call (cost + latency)
- ❌ Hallucinated hypothetical answer may mislead retrieval
- ❌ Not always better than standard retrieval

**Best for:**
- Complex, open-ended questions
- Domain-specific queries with jargon
- When you have budget for extra LLM call

---

## 7. Complex Metadata Filtering

### AND/OR/NOT Logic

In [15]:
# Create filters with complex logic

# Example 1: Papers from 2017-2019 AND category='transformers'
filters_and = MetadataFilters(
    filters=[
        MetadataFilter(key="year", operator=FilterOperator.GTE, value=2017),
        MetadataFilter(key="year", operator=FilterOperator.LTE, value=2019),
        MetadataFilter(key="category", operator=FilterOperator.EQ, value="transformers"),
    ],
    condition=FilterCondition.AND,
)

retriever_and = fusion_index.as_retriever(
    similarity_top_k=5,
    filters=filters_and,
)

results_and = retriever_and.retrieve("neural network architecture")
print("Filter: year >= 2017 AND year <= 2019 AND category='transformers'")
print(f"Results: {len(results_and)}")
for node in results_and:
    print(f"  - {node.metadata.get('title')} ({node.metadata.get('year')})")

Filter: year >= 2017 AND year <= 2019 AND category='transformers'
Results: 4
  - Attention Is All You Need (2017)
  - Attention Is All You Need (2017)
  - Attention Is All You Need (2017)
  - Attention Is All You Need (2017)


In [16]:
# Example 2: High citations (>10000) OR recent (year >= 2020)
filters_or = MetadataFilters(
    filters=[
        MetadataFilter(key="citations", operator=FilterOperator.GT, value=10000),
        MetadataFilter(key="year", operator=FilterOperator.GTE, value=2020),
    ],
    condition=FilterCondition.OR,
)

retriever_or = fusion_index.as_retriever(
    similarity_top_k=10,
    filters=filters_or,
)

results_or = retriever_or.retrieve("language models")
print("\nFilter: citations > 10000 OR year >= 2020")
print(f"Results: {len(results_or)}")
unique_titles = set()
for node in results_or:
    title = node.metadata.get('title')
    if title not in unique_titles:
        unique_titles.add(title)
        print(f"  - {title} ({node.metadata.get('year')}, {node.metadata.get('citations')} citations)")


Filter: citations > 10000 OR year >= 2020
Results: 6
  - Retrieval-Augmented Generation (RAG) (2020, None citations)
  - BERT: Pre-training of Deep Bidirectional Transformers (2019, 65000 citations)
  - Attention Is All You Need (2017, 85000 citations)


### Advanced Filtering Patterns

**Filter Operators:**
- `EQ`: Equal
- `NE`: Not equal
- `GT`: Greater than
- `GTE`: Greater than or equal
- `LT`: Less than
- `LTE`: Less than or equal
- `IN`: In list
- `NIN`: Not in list
- `CONTAINS`: String contains

**Filter Conditions:**
- `AND`: All filters must match
- `OR`: At least one filter must match

---

## 8. Retrieval Evaluation

### Measuring Retrieval Quality

In [17]:
# Define evaluation queries with known relevant documents
eval_queries = [
    {
        "query": "How does self-attention work?",
        "relevant_titles": ["Attention Is All You Need"],
    },
    {
        "query": "What is masked language modeling?",
        "relevant_titles": ["BERT: Pre-training of Deep Bidirectional Transformers"],
    },
    {
        "query": "How does retrieval augmented generation work?",
        "relevant_titles": ["Retrieval-Augmented Generation (RAG)"],
    },
]

def evaluate_retrieval(retriever, queries, k=3):
    """Calculate precision@k and recall@k."""
    precisions = []
    recalls = []
    
    for item in queries:
        query = item["query"]
        relevant = set(item["relevant_titles"])
        
        # Retrieve
        nodes = retriever.retrieve(query)
        retrieved_titles = [n.metadata.get("title") for n in nodes[:k]]
        retrieved = set(retrieved_titles)
        
        # Calculate metrics
        true_positives = len(relevant & retrieved)
        precision = true_positives / k if k > 0 else 0
        recall = true_positives / len(relevant) if relevant else 0
        
        precisions.append(precision)
        recalls.append(recall)
    
    return {
        "precision@k": sum(precisions) / len(precisions),
        "recall@k": sum(recalls) / len(recalls),
    }

# Evaluate different retrievers
base_retriever_eval = fusion_index.as_retriever(similarity_top_k=3)
metrics = evaluate_retrieval(base_retriever_eval, eval_queries, k=3)

print("Retrieval Evaluation Metrics:")
print(f"  Precision@3: {metrics['precision@k']:.2%}")
print(f"  Recall@3: {metrics['recall@k']:.2%}")

Retrieval Evaluation Metrics:
  Precision@3: 33.33%
  Recall@3: 100.00%


### 🎯 ML Engineering Note: Retrieval Metrics

**Key Metrics:**

1. **Precision@K**: Of K retrieved docs, how many are relevant?
   - Formula: (relevant docs in top K) / K
   - High precision = low false positives

2. **Recall@K**: Of all relevant docs, how many are in top K?
   - Formula: (relevant docs in top K) / (total relevant docs)
   - High recall = low false negatives

3. **MRR (Mean Reciprocal Rank)**: Where does first relevant doc appear?
   - Formula: 1 / rank of first relevant doc
   - Higher = relevant docs appear earlier

4. **NDCG (Normalized Discounted Cumulative Gain)**: Weighted relevance
   - Considers graded relevance and position
   - Gold standard for ranking evaluation

**Production Monitoring:**
- Track precision@k and recall@k over time
- Set alerts for metric degradation
- A/B test retrieval strategies
- Collect user feedback (clicks, dwell time)

---

## 9. Summary: What You Learned

### ✅ Completed Learning Objectives

1. **RecursiveRetriever**: Hierarchical retrieval for structured documents
2. **QueryFusionRetriever**: Combined multiple retrievers with RRF
3. **Custom Retrievers**: Built citation-aware re-ranking
4. **Query Transformation**: Applied HyDE for better retrieval
5. **Metadata Filtering**: Implemented complex AND/OR/NOT logic
6. **Evaluation**: Measured retrieval quality with precision and recall

### Key Concepts Mastered

- **RecursiveRetriever**: Two-stage retrieval (summary → details)
- **QueryFusionRetriever**: Reciprocal Rank Fusion (RRF)
- **Custom retrievers**: BaseRetriever subclass, custom scoring
- **HyDE**: Hypothetical document embeddings
- **Metadata filters**: FilterOperator, FilterCondition
- **Evaluation metrics**: Precision@K, Recall@K, MRR

### Next Steps

In **Notebook 5: Hybrid Search & Reranking**, you'll learn:
- Dense vs sparse vectors (BM25)
- Hybrid search with Qdrant
- Alpha parameter tuning
- Reranking models (Cohere, cross-encoders)
- Custom query engines
- Production patterns and optimization

---

## 🎯 Practice Exercises

1. **Hierarchical Documents**: Create a 3-level hierarchy (book → chapter → section) with RecursiveRetriever
2. **Query Fusion**: Combine 3+ retrievers with different strategies. Measure improvement.
3. **Custom Retriever**: Build a time-decay retriever that favors recent documents
4. **HyDE Comparison**: Test HyDE on 10 queries. When does it help vs hurt?
5. **Evaluation Suite**: Build comprehensive eval with MRR and NDCG

---

## Additional Resources

- **Retrievers**: https://docs.llamaindex.ai/en/stable/module_guides/querying/retriever/
- **Query Transformations**: https://docs.llamaindex.ai/en/stable/examples/query_transformations/
- **Metadata Filtering**: https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/